In [1]:
import os
from pathlib import Path

# Document folder

In [ ]:
doc_dir = "doc_folder_2"

files = list(Path(doc_dir).glob("*"))
print(files)

[PosixPath('doc_folder_1/Public003.pdf'), PosixPath('doc_folder_1/Public004.pdf')]


In [3]:
file = files[0]
print(file)

doc_folder_1/Public003.pdf


# Basic usage

Using `DocumentConverter` to process a document.

In [1]:
import os, sys
sys.path.append("..")

In [2]:
import os
import logging
from io import BytesIO
from pathlib import Path
import subprocess

import docling
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline


from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling_core.types.io import DocumentStream
from docling.datamodel.pipeline_options import TableFormerMode
from docling.datamodel.accelerator_options import AcceleratorDevice, AcceleratorOptions
from docling.datamodel.document import ConversionResult
from docling.datamodel.pipeline_options_vlm_model import ApiVlmOptions, ResponseFormat
from docling.datamodel.pipeline_options import (
    VlmPipelineOptions,
)
from docling.pipeline.vlm_pipeline import VlmPipeline

from PIL import Image
import torch
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
import time
import socket

# Step 1 — Set env vars before starting server
os.environ["OLLAMA_FLASH_ATTENTION"] = "true"
os.environ["OLLAMA_LOW_VRAM"] = "false"
os.environ["OLLAMA_NUM_PARALLEL"] = "4"
os.environ["OLLAMA_CONTEXT_LENGTH"] = "2048"

# Step 2 — Start Ollama server with these variables

def is_ollama_running():
	sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
	result = sock.connect_ex(('localhost', 11434))
	sock.close()
	return result == 0

if not is_ollama_running():
	ollama_proc = subprocess.Popen(["ollama", "serve"], env=os.environ)
	time.sleep(5)  # wait for Ollama to be ready
else:
	print("Ollama is already running")


# MODEL_PATH = "benhaotang/Nanonets-OCR-s:latest"  # local HF model
MODEL_PATH = "benhaotang/Nanonets-OCR-s:q4_k_m"
os.environ['TRANSFORMERS_VERBOSITY'] = 'info'


def create_vlm_options(model:str, prompt:str):
    options = ApiVlmOptions(
        url="http://localhost:11434/v1/chat/completions",  # the default Ollama endpoint # type: ignore
        params=dict(
            model=model,
        ),
        prompt=prompt,
        timeout=350,
        scale=1.0,
        response_format=ResponseFormat.MARKDOWN,
    )

    return options


# ===========================
# 2️⃣ Configure Docling
# ===========================
def doc_converter():
    logging.basicConfig(level=logging.INFO)
    system_prompt = """
        Hãy trích xuất toàn bộ văn bản từ tài liệu ở trên giống như cách bạn đọc nó một cách tự nhiên.
        Trả về các bảng dưới dạng mã HTML.
        Trả về các phương trình dưới dạng biểu diễn LaTeX.

        Nếu trong tài liệu có hình ảnh nhưng không có chú thích, hãy thêm một mô tả ngắn cho hình ảnh đó bên trong thẻ <img></img>;
        nếu hình ảnh đã có chú thích, hãy đặt chú thích đó bên trong thẻ <img></img>.

        Dấu watermark nên được đặt trong thẻ <watermark></watermark>.
        Số trang nên được đặt trong thẻ <page_number></page_number>.
        Ví dụ: <page_number>14</page_number> hoặc <page_number>9/22</page_number>.

        Ưu tiên sử dụng ký hiệu ☐ và ☑ cho các ô kiểm (checkbox).

        QUAN TRỌNG: Luôn luôn trả lời bằng **Tiếng Việt**.
    """

    #? run remote Ollama, Huggingface model
    pdf_options = VlmPipelineOptions(
        enable_remote_services=True  # required when calling remote VLM endpoints
    )
    pdf_options.accelerator_options = AcceleratorOptions(
        num_threads=12, device=AcceleratorDevice.AUTO
    )
    pdf_options.vlm_options = create_vlm_options(
        model=MODEL_PATH,
        prompt=system_prompt,
    )

    converter = DocumentConverter(
        # allowed_formats=[
        #     InputFormat.PDF,
        #     InputFormat.IMAGE,
        #     InputFormat.DOCX,
        #     InputFormat.HTML,
        #     InputFormat.PPTX,
        # ],  # whitelist formats, non-matching files are igno

        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pdf_options,
                pipeline_cls=VlmPipeline,
            )
        }
    )

    return converter


# PDF → Markdown pipeline
def extract_text_to_markdown(input_dir: str, output_dir: str):
    output_dir = Path(output_dir)  # type: ignore
    output_dir.mkdir(parents=True, exist_ok=True)  # type: ignore

    data_folder = 'D:\\VIETTEL_RACE\\qa-mcq-rag\\mcqaRAG_p2\\private-test-input'
    file_names = os.listdir(data_folder)
    file_list = [f'{data_folder}/{file_name}' for file_name in file_names]

    batch_size = 3
    converter = doc_converter()

    # Process files in batches
    for i in range(0, len(file_list), batch_size):
        batch_names = file_names[i:i+batch_size]
        batch_paths = file_list[i:i+batch_size]

        print(f"Processing batch {i//batch_size + 1}: {batch_names}")

        for file_name, file_path in zip(batch_names, batch_paths):
            try:
                conv_result = converter.convert(file_path)
                with open(f"{output_dir}/{file_name[:-4]}.md", "w", encoding='utf-8') as f:
                    f.write(conv_result.document.export_to_markdown())
                print(f"✓ Completed: {file_name}")
            except Exception as e:
                print(f"❌ Error processing {file_name}: {e}")


Ollama is already running


In [ ]:
if __name__ == '__main__':
    start_time = time.perf_counter()  # Use perf_counter for higher precision
    extract_text_to_markdown('private-test-input', '../scratch')

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.4f} seconds")
    ollama_proc.terminate() # type: ignore

2025-10-19 01:34:24,219 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-19 01:34:24,259 - INFO - Going to convert document batch...
2025-10-19 01:34:24,260 - INFO - Initializing pipeline for VlmPipeline with options hash 309ed9ea03de69421ff1373882da7d51


Processing batch 1: ['Public001.pdf', 'Public002.pdf', 'Public003.pdf']


2025-10-19 01:34:24,710 - INFO - Loading plugin 'docling_defaults'
2025-10-19 01:34:24,710 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-10-19 01:34:24,710 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-19 01:34:24,710 - INFO - Processing document Public001.pdf
2025-10-19 01:36:04,880 - INFO - Finished converting document Public001.pdf in 100.66 sec.
2025-10-19 01:36:04,888 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-19 01:36:04,888 - INFO - Going to convert document batch...
2025-10-19 01:36:04,888 - INFO - Processing document Public002.pdf


✓ Completed: Public001.pdf


2025-10-19 01:36:56,096 - INFO - Finished converting document Public002.pdf in 51.20 sec.
2025-10-19 01:36:56,115 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-19 01:36:56,116 - INFO - Going to convert document batch...
2025-10-19 01:36:56,116 - INFO - Processing document Public003.pdf


...

✓ Completed: Public066.pdf


2025-10-19 05:21:13,817 - INFO - Finished converting document Public067.pdf in 24.61 sec.
2025-10-19 05:21:13,833 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-19 05:21:13,833 - INFO - Going to convert document batch...
2025-10-19 05:21:13,833 - INFO - Processing document Public068.pdf


✓ Completed: Public067.pdf


2025-10-19 06:31:42,591 - INFO - Finished converting document Public068.pdf in 4228.78 sec.
2025-10-19 06:31:42,748 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-19 06:31:42,748 - INFO - Going to convert document batch...
2025-10-19 06:31:42,754 - INFO - Processing document Public069.pdf


✓ Completed: Public068.pdf
Processing batch 25: ['Public069.pdf', 'Public070.pdf', 'Public071.pdf']


# Annotate images with ollama

In [ ]:
import sys
from pathlib import Path

# Thêm đường dẫn project vào sys.path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc.base import ImageRefMode
from src.config import config

In [ ]:
def convert_with_image_annotation(input_doc_path):
    # Đọc cấu hình từ config.yaml
    base_url = config.get("model", "url", default="http://localhost:11434/v1")
    vision_model = config.get("model", "vision_model", default="")
    text_model = config.get("model", "text_generation", default="")
    picture_prompt = config.get("document", "picture_description", "prompt_picture_description",
                                default="Miêu tả chi tiết hình ảnh sau bằng một đoạn văn.")
    pd_enabled = config.get("document", "picture_description", "enabled", default=True)
    image_scale = config.get("document", "image_resolution_scale", default=2)

    # Nếu tắt picture description, không dùng remote API
    if not pd_enabled:
        pipeline_options = PdfPipelineOptions(
            images_scale=image_scale,
            generate_picture_images=False,
            do_picture_description=False,
            picture_description_options=None,
            enable_remote_services=False,
        )
    else:
        # Dùng vision_model nếu có, fallback sang text_model
        model_name = vision_model if vision_model else text_model

        # Chuẩn hóa URL cho Ollama OpenAI-compatible API
        if not base_url.endswith("/chat/completions") and not base_url.endswith("/v1/chat/completions"):
            if base_url.endswith("/v1"):
                api_url = f"{base_url}/chat/completions"
            else:
                api_url = f"{base_url}/v1/chat/completions"
        else:
            api_url = base_url

        picture_desc_api_option = PictureDescriptionApiOptions(
            url=api_url,
            prompt=picture_prompt,
            params={"model": model_name},
            timeout=60,
        )

        pipeline_options = PdfPipelineOptions(
            images_scale=image_scale,
            generate_picture_images=True,
            do_picture_description=True,
            picture_description_options=picture_desc_api_option,
            enable_remote_services=True,
        )

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    conv_res = converter.convert(source=input_doc_path)
    return conv_res

In [9]:
result = convert_with_image_annotation(file)

2025-10-17 16:54:30,127 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-17 16:54:30,132 - INFO - Going to convert document batch...
2025-10-17 16:54:30,133 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 68e059f3e75bb40d4b4d82a1df76f3d7
2025-10-17 16:54:30,133 - INFO - Accelerator device: 'mps'


KeyboardInterrupt: 

In [ ]:
print(result.document.export_to_markdown(mark_annotations = True, include_annotations=True))

<!-- image -->

## 1. L ờ i m ở đầ u

Bài toán nhận diện biển số xe Việt Nam là một bài toán không còn mới, đã được phát triển dựa trên các phương pháp xử lý ảnh truyền thống và cả những kỹ thuật mới sử dụng Deep Learning. Trong bài toán này tôi chỉ phát triển bài toán phát hiện biển số (một phần trong bài toán nhận diện biển số) dựa trên thuật toán YOLO -Tinyv4 với mục đích:

- Hướng dẫn chuẩn bị dữ liệu cho bài toán Object Detection.
- Hướng dẫn huấn luyện YOLO -TinyV4 dùng darknet trên Google Colab.

## 2. Chu ẩ n b ị d ữ li ệ u

## 2.1 Đánh giá bộ d ữ li ệ u

Trong bài viết tôi sử dụng bộ dữ liệu biển số xe máy Việt Nam chứa 1750 ảnh, bạn đọc có thể tải tại đây .

Hình 14.1: Ảnh biển số trong bộ dữ liệu

<!--<annotation kind="description">-->Bái có cảm ảng cấp trong mạnh trị của một vị trí chuyển lại phúc vào thành công tất cả.<!--<annotation/>-->

<!-- image -->

Ảnh biển số xe được trong bộ dữ liệu được chụp từ một camera tại vị trí kiểm soát xe ra vào trong hầm. Do vậy:

- Kích 

# Save to MD with external referenced images

In [ ]:
def export_function_md_with_image_ref(conv_res, output_path:str, replace_blank:str="_"):

    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem.replace(" ", replace_blank)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED, include_annotations=True)

In [ ]:
export_function_md_with_image_ref(result, "outputs")